<a href="https://colab.research.google.com/github/HelloShagun/useful_libs/blob/main/examples/end_to_end_Fish_Ponds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fish pond maching

In [8]:
!pip install contextily
import os, sys, importlib
import numpy as np
import pandas as pd


In [ ]:
!git clone https://github.com/HelloShagun/useful_libs
from google.colab import output
output.enable_custom_widget_manager()
import sys
sys.path.append("/content/useful_libs/")


##### collab
from google.colab import drive
drive.mount('/content/drive')


In [10]:
from lake_maps_comparison.io_stack import read_stack, get_band_mask
from lake_maps_comparison.morphology import MorphologyConfig, clean_mask
from lake_maps_comparison.objects import ObjectConfig, label_and_stats
from lake_maps_comparison.matching import MatchConfig, union_masks, match_objects_labels_to_mask
from lake_maps_comparison.vectorize import labels_to_geodataframe
from lake_maps_comparison.wayback_export import export_wayback_panels, ExportStyle

In [11]:


# ---- 2) USER CONFIG (edit these) ----
TIF_PATH = "/content/drive/MyDrive/Alpha_Earth/Fish_Lakes/Water_Mask_and_Similarity_Final.tif"  # <-- your stack
OUT_DIR  = "/content/drive/MyDrive/Alpha_Earth/Fish_Lakes/Output"
os.makedirs(OUT_DIR, exist_ok=True)

BASE_PRODUCTS = ["SARL_Water", "ESA_Water", "JRC_Water", "DW_Water", "HydroLAKES"]
AE_MAIN = "AE_ge_90"


tif_file_path= '/content/drive/MyDrive/Alpha_Earth/Fish_Lakes/Water_Mask_and_Similarity_Final.tif'
a,b = read_stack(tif_file_path)


In [ ]:
import numpy as np
import cv2
import leafmap
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from shapely.geometry import shape, mapping
from rasterio.features import shapes
from affine import Affine

# --- 1. USER HYPERPARAMETERS ---
# Adjust these based on your images:
LOW_THRESH = 0.68      # Lower this if ponds are "shrunken" like in image_5443cb.jpg
HIGH_THRESH = 0.96     # Higher threshold to ensure separation in image_54afb6.jpg
MIN_PEAK_DIST = 4      # Smallest distance to keep ponds separate
SMOOTH_FACTOR = 0.04   # Increase this (e.g. 0.04) to fix jagged edges in image_21ddcf.jpg

# --- 2. DATA SETUP ---
sim_band = a[10, :, :]
transform = Affine(8.983152841195215e-05, 0.0, 110.724994447646,
                   0.0, -8.983152841195215e-05, 19.742005504509493)

# --- 3. THE SEGMENTATION ENGINE ---
# Create masks for Hysteresis
mask_area = (sim_band > LOW_THRESH).astype(np.uint8)
sure_centers = (sim_band > HIGH_THRESH).astype(np.uint8)

# Marker Detection (Prevents blobs in image_54afb6.jpg)
dist = cv2.distanceTransform(sure_centers, cv2.DIST_L2, 5)
peaks = peak_local_max(dist, min_distance=MIN_PEAK_DIST, labels=mask_area)

markers_mask = np.zeros(sim_band.shape, dtype=bool)
markers_mask[tuple(peaks.T)] = True
markers, _ = ndi.label(markers_mask)

# Watershed (Preserves separations/bridges)
labels = watershed(-sim_band, markers, mask=mask_area)

# --- 4. VECTORIZATION & SMOOTHING ---
features = []
for geom, val in shapes(labels.astype(np.int32), mask=(labels > 0), transform=transform):
    # Fix jagged edges (image_21ddcf.jpg) using Shapely simplification
    poly_geom = shape(geom)

    # Simplify geometry to remove "staircase" pixels
    # The tolerance is in degrees; 0.0001 is roughly 11 meters
    simplified = poly_geom.simplify(8.98e-05 * SMOOTH_FACTOR, preserve_topology=True)

    if simplified.area > (8.0697e-09 * 10):
        features.append({
            "type": "Feature",
            "geometry": mapping(simplified),
            "properties": {"id": int(val)}
        })

geojson_data = {"type": "FeatureCollection", "features": features}

# --- 6. MAP (WITH METADATA FIX) ---
m = leafmap.Map(center=[19.742, 110.725], zoom=16)

# Specific 2021 Wayback Tile
wayback_2021 = 'https://wayback.maptiles.arcgis.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}?date=20210210'
m.add_tile_layer(wayback_2021, name='ESRI 2021', attribution='Esri')

# Add your segmented ponds
m.add_geojson(geojson_data, layer_name="Textured Rectangular Ponds",
              style={'color': '#FF00FF', 'weight': 2, 'fillOpacity': 0.1})

# FIX FOR THE RENDERER ERROR:
# Option A: If you just want to view it in the notebook
m.to_html(outfile="pond_map.html") # This saves a clean HTML file you can open
m # Display the map as usual
m